In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import os
import math

import matplotlib.pyplot as plt
#import seaborn as sns

In [ ]:
#options = pd.read_csv(train_files+'/options.csv')
#sec_stock_price = pd.read_csv(train_files+'/secondary_stock_prices.csv')
stock_price = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
#trades = pd.read_csv(train_files+'/trades.csv')
#financials = pd.read_csv(train_files+'/financials.csv')

In [ ]:
stock_price.head()

In [ ]:
stock_price['AdjClose'] = stock_price['Close'] * stock_price['AdjustmentFactor']
stock_price['Date'] = pd.to_datetime(stock_price['Date'])

In [ ]:
df_1301 = stock_price[stock_price['SecuritiesCode']==1301].set_index('Date')
df_1301.head()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
def create_dataset(dataset, look_back = 1):
    dataX, dataY = [],[]
    for i in range(len(dataset) - look_back -1):
        a = dataset[i:(i+look_back),0]
        dataX.append(a)
        dataY.append(dataset[i+look_back,0])
    return np.array(dataX), np.array(dataY)

In [ ]:
dataset = df_1301['AdjClose'].dropna().values.reshape(-1,1)
dataset = dataset.astype('float32')

In [ ]:
scaler = MinMaxScaler(feature_range = (0,1))
dataset = scaler.fit_transform(dataset)

In [ ]:
rate = 0.9
train_size = int(len(dataset)*rate)
test_size = len(dataset)-train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train,look_back)
testX, testY = create_dataset(test,look_back)

In [ ]:
train_X = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
test_X = np.reshape(testX,(testX.shape[0],1, testX.shape[1]))

In [ ]:
train_X.shape, trainY.shape

In [ ]:
#Modeling
model = Sequential()
model.add(LSTM(4, input_shape = (train_X.shape[1],look_back))) #4个神经元
model.add(Dense(1)) # 1 个隐藏层
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.fit(train_X, trainY, epochs=100, batch_size = 32, verbose=2) # 训练100轮

In [ ]:
accuracy = model.evaluate(train_X, trainY)

In [ ]:
trainPredict = model.predict(train_X)
testPredict = model.predict(test_X)

In [ ]:
train_Predict = scaler.inverse_transform(trainPredict)
train_Y = scaler.inverse_transform([trainY])

test_Predict = scaler.inverse_transform(testPredict)
test_Y = scaler.inverse_transform([testY])

In [ ]:
test_Y[0][1], test_Predict[1][0]

In [ ]:
trainScore = math.sqrt(mean_squared_error(train_Y[0], train_Predict[:,0]))
print('Train Score is %.2f rmse' %(trainScore))

testScore = math.sqrt(mean_squared_error(test_Y[0][0:], test_Predict[:,0]))
print('Test Score is %.2f rmse' %(testScore))

In [ ]:
plt.figure(figsize = (12,7))
plt.plot(test_Predict[1:], color = '#FDAC53',marker = 'o', label = 'Prediction')
plt.plot(np.reshape(test_Y, (test_Y.shape[1], test_Y.shape[0])), color = 'blue', label = 'Actual')
plt.legend()
plt.show()

In [ ]:
def LSTM_model(dataX, dataY):
    model = Sequential()
    model.add(LSTM(4, input_shape = (dataX.shape[1],look_back)))
    model.add(Dense(1)) 
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    model.fit(dataX, dataY, epochs=100, batch_size = 32, verbose=2)

In [ ]:
pred_list = []
for i in stock_price['SecuritiesCode'].unique():
    print('Security Code:', i)
    df = stock_price[stock_price['SecuritiesCode']==i].set_index('Date')
    df_close = df['AdjClose'].dropna()
    dataset = df_close.values.reshape(-1,1)
    dataset = dataset.astype('float32')
    scaler = MinMaxScaler(feature_range = (0,1))
    dataset = scaler.fit_transform(dataset)
    
    look_back = 1
    dataX, dataY = create_dataset(dataset,look_back)
    X = np.reshape(dataX, (dataX.shape[0], 1, dataX.shape[1]))
    LSTM_model(X, dataY)
    X_ = dataset[- look_back:]  # last available input sequence
    X_ = X_.reshape(1, look_back, 1)

    Y_ = model.predict(X_).reshape(-1, 1)
    Y_ = scaler.inverse_transform(Y_)
    
    actual = df_close[-1]
    predict = Y_[0][0]
    
    return_ = predict - actual
    pred_list.append([i,return_])

In [ ]:
df_result = pd.DataFrame(data =pred_list,columns = ['Code','Return'])
df_result.head()

In [ ]:
#from pandas.tseries.offsets import BDay

In [ ]:
#last_date = stock_price.Date.max()
#next_BDay = last_date+BDay(1)

In [ ]:
#df_result.loc[:,'Date'] = next_BDay 

In [ ]:
df_result.loc[:,'Rank2'] = df_result['Return'].rank(method = 'first',ascending = False).astype(int)

In [ ]:
df_final = df_result[['Date','Code','Rank2']].reset_index(drop=True)

In [ ]:
df_final.head()

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    sample_prediction_df = sample_prediction.merge(df_final, how = 'left', left_on ='SecuritiesCode', right_on = 'Code')
    final = sample_prediction_df[['Date','SecuritiesCode','Rank2']].rename(columns = {'Rank2':'Rank'})
    env.predict(final)   # register your predictions